In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [3]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import keras
from keras import regularizers
from glob import glob
from tensorflow.python.keras.layers import Input, Dense

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
IMAGE_SIZE = [224, 224]

In [7]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
#vgg.summary()

58892288/58889256 [==============================] - 1s 0us/step


In [8]:
for layer in vgg16.layers:
  layer.trainable = False

In [9]:
x = Flatten()(vgg16.output)
prediction = Dense(1, activation='sigmoid')(x)
model = Model(inputs=vgg16.input, outputs=prediction)

In [10]:
#model.summary()

In [11]:
for layer in model.layers:
    layer.kernel_regularizer = keras.regularizers.l1_l2(0.001)

In [12]:
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [13]:
train_path = '/content/drive/MyDrive/real vs fake/NEW_TRAIN_10K'
valid_path = '/content/drive/MyDrive/real vs fake/test'

In [14]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.3,
                                   zoom_range=0.3,
                                   rotation_range=90,
                                   width_shift_range=0.3,
                                   #fill_mode='wrap',
                                   height_shift_range=0.3,
                                   #channel_shift_range=150.0,
                                   brightness_range=(0.1, 0.5),
                                   vertical_flip=True,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [15]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/real vs fake/NEW_TRAIN_10K',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 color_mode='rgb',
                                                 class_mode = 'binary')

Found 10000 images belonging to 2 classes.


In [16]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/real vs fake/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            color_mode='rgb',
                                            class_mode = 'binary')

Found 3000 images belonging to 2 classes.


In [17]:
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/20
313/313 [==============================] - 3484s 11s/step - loss: 0.3407 - accuracy: 0.8466 - val_loss: 0.8776 - val_accuracy: 0.7053
Epoch 2/20
313/313 [==============================] - 158s 504ms/step - loss: 0.2373 - accuracy: 0.9014 - val_loss: 0.9693 - val_accuracy: 0.7203
Epoch 3/20
313/313 [==============================] - 161s 513ms/step - loss: 0.2031 - accuracy: 0.9183 - val_loss: 0.1752 - val_accuracy: 0.9387
Epoch 4/20
313/313 [==============================] - 158s 506ms/step - loss: 0.1830 - accuracy: 0.9282 - val_loss: 0.4350 - val_accuracy: 0.8687
Epoch 5/20
313/313 [==============================] - 160s 509ms/step - loss: 0.1769 - accuracy: 0.9300 - val_loss: 0.4340 - val_accuracy: 0.8717
Epoch 6/20
313/313 [==============================] - 161s 514ms/step - loss: 0.1692 - accuracy: 0.9345 - val_loss: 0.2511 - val_accuracy: 0.9250
Epoch 7/20
313/313 [==============================] - 164s 523ms/step - loss: 0.1648 - accuracy: 0.9330 - val_loss: 0.3802 - 

In [1]:
import matplotlib.pyplot as plt

In [2]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

NameError: ignored